# Sentinel-2 

In [ ]:
from dotenv import load_dotenv

load_dotenv()

import os

import fsspec
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import numpy as np
import odc.stac
import pystac
import rioxarray
import shapely
import stac_geoparquet
import xarray as xr
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

## Zoom to your area of interest

In [ ]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = m.center = (53.19, 4.9)
m.zoom = 12
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.757, 53.108, 5.042, 53.272))
west, south, east, north = list(roi.total_bounds)

In [ ]:
from coastpy.utils.grid import read_coastal_grid

In [ ]:
grid = read_coastal_grid(9, "10000m")

## Sentinel 2 Composite

In [ ]:
from coastpy.eo.collection import S2Collection
s2 = (
    S2Collection(stac_cfg={})
    .search(
        roi,
        date_range="2024-06-01/2024-08-31",
        query={"eo:cloud_cover": {"lt": 30}},
    )
    .load(
        bands=["red", "green", "blue"],
        patch_url=pc.sign,
        resampling=resampling,
        dtype="float32",
        resolution=10,
        crs="utm",
    )
    .mask_and_scale(
        mask_nodata=True,
        scale=True,
        scale_factor=0.0001,
    )
    .execute(compute=False)
)